# Experimenting for A1
- Figuring out how to solve the stuff

TODO: Refactor into a script

In [106]:
import cv2
import imutils
import functools
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List

In [63]:
# Helper functions
def to_gray(img): 
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.uint8)

def calc_color_hist(img, key=None):         
    return cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    
def create_norm_hist(img: np.ndarray, key=None) -> np.ndarray: 
    hist = calc_color_hist(img, key=key)
    return cv2.normalize(hist, hist).flatten()

def read_img(path: Path) -> np.ndarray: 
    return cv2.imread(str(path))

def process_img(img_path: Path):
    img = read_img(img_path)
    return create_norm_hist(img)

def compare_hists(source_hist, candidate_hist):
    return cv2.compareHist(source_hist, candidate_hist, cv2.HISTCMP_CHISQR)

In [2]:
DATA_DIR = Path("../../../CDS-VIS/flowers")
assert DATA_DIR.exists()

In [64]:
all_flowers = list(DATA_DIR.glob("*.jpg"))
test_flower = all_flowers[0]
all_flowers.remove(test_flower)
assert test_flower not in all_flowers

In [77]:
source_img = read_img(test_flower)
testy = create_norm_hist(source_img)

In [36]:
cv2.compareHist(testy, testy, cv2.HISTCMP_CHISQR)

0.0

In [46]:
source_hist = process_img(test_flower)



In [67]:
dist_df = pd.DataFrame({"dist": 0}, index=all_flowers)

In [68]:
dist_df

,dist
../../../CDS-VIS/flowers/image_0772.jpg,0
../../../CDS-VIS/flowers/image_1307.jpg,0
../../../CDS-VIS/flowers/image_0127.jpg,0
../../../CDS-VIS/flowers/image_0479.jpg,0
../../../CDS-VIS/flowers/image_1250.jpg,0
...,...
../../../CDS-VIS/flowers/image_0682.jpg,0
../../../CDS-VIS/flowers/image_1342.jpg,0
../../../CDS-VIS/flowers/image_1025.jpg,0
../../../CDS-VIS/flowers/image_1061.jpg,0


In [138]:
for i, flower in enumerate(all_flowers):
    candidate_hist = process_img(flower)
    dist_df.loc[flower, "dist"] += compare_hists(source_hist, candidate_hist)
    if i % 200 == 0: 
        print(f"processed {i} imgs!")

processed 0 imgs!
processed 200 imgs!
processed 400 imgs!
processed 600 imgs!
processed 800 imgs!
processed 1000 imgs!
processed 1200 imgs!


In [153]:
closest_imgs = dist_df.nsmallest(3, "dist")
output_dict = {"source": [test_flower.name], "1st": [closest_imgs.index[0].name], "2nd": [closest_imgs.index[1].name], "3rd": [closest_imgs.index[2].name]}
pd.DataFrame.from_dict(output_dict)

,source,1st,2nd,3rd
0,image_1079.jpg,image_0626.jpg,image_0228.jpg,image_0077.jpg


In [79]:
def add_text(img, text):
    new_img = img.copy()
    # setup text
    font = cv2.FONT_HERSHEY_SIMPLEX

    # get boundary of this text
    textsize = cv2.getTextSize(text, font, 1, 2)[0]

    # get coords based on boundary
    textX = (img.shape[1] - textsize[0]) // 2
    textY = (img.shape[0] + textsize[1]) // 2

    # add text centered on image
    cv2.putText(new_img, text, (textX, textY ), font, 1, (255, 255, 255), 2)
    
    return new_img

In [80]:
cv2.imwrite("test_img.jpg", add_text(source_img, "testy"))

True

In [82]:
def format_dist(dist: float) -> str:
    return f"dist:{dist: .2f}"

format_dist(2.132456)

'dist: 2.13'

In [132]:
def resize_square(img, size=300):
    return cv2.resize(img, dsize=(size, size))

In [133]:
dist_img_list = [resize_square(add_text(source_img, "SOURCE"))]
for filename, row in dist_df.nsmallest(3, "dist").iterrows():
    img = resize_square(read_img(filename))
    assert img.shape[1] == 300
    dist_img_list.append(add_text(img, format_dist(row["dist"])))

In [136]:
import math 
def arrange_square(img_list: List[np.ndarray], img_dim=300) -> np.ndarray: 
    """ Adapted from https://stackoverflow.com/a/52283965 """ 
    if len(img_list) not in [i**2 for i in range(10)]: 
        raise ValueError("List must have square number of elements")
    
    canvas_shape = math.isqrt(len(img_list))
    imgmatrix = np.zeros((canvas_shape*img_dim, canvas_shape*img_dim, 3), np.uint8)
    #Prepare an iterable with the right dimensions
    positions = itertools.product(range(canvas_shape), range(canvas_shape))

    for (y_i, x_i), img in zip(positions, img_list):
        x = x_i * img_dim
        y = y_i * img_dim
        imgmatrix[y:y+img_dim, x:x+img_dim, :] = img
    return imgmatrix
        

In [154]:
import itertools
testy = arrange_square(dist_img_list)

In [137]:
all_imgs = np.vstack(dist_img_list)
cv2.imwrite("testy.png", testy)

True